In [14]:
import os

from pytube import Playlist
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Neo4jVector
from langchain.document_loaders import YoutubeLoader

from langchain.chat_models import ChatOpenAI
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [15]:
# playlist_url = "https://www.youtube.com/watch?v=hCMv4UJo--4&list=PL9Hl4pk2FsvWPcphew_GbLjCWvMpmh4mV&ab_channel=Neo4j"
# playlist = Playlist(playlist_url)
# video_ids = [_v.split('v=')[-1] for _v in playlist.video_urls]
# Process only one video
video_ids = ['qYaXwV9ZNr4']

In [16]:
# Init text splitter with chunk size 512 (https://www.pinecone.io/learn/chunking-strategies/)
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=512, chunk_overlap=20)

In [17]:
from env import setup_env
setup_env()

In [18]:
# Read their captions and process it into documents with above defined text splitter
documents = []
for video_id in video_ids:
    loader = YoutubeLoader(video_id=video_id)
    documents.append(loader.load_and_split(text_splitter=text_splitter)[0])

In [19]:
# Contruct vector
neo4j_vector = Neo4jVector.from_documents(
    embedding=OpenAIEmbeddings(),
    documents=documents,
    url=os.environ['NEO4J_URI'],
    username=os.environ['NEO4J_USERNAME'],
    password=os.environ['NEO4J_PASSWORD']
)

In [21]:
# Prepare Q&A object
chat = ChatOpenAI()
chat_mem_history = ChatMessageHistory(sesion_id="1")
mem = ConversationBufferMemory(memory_key="chat_history", chat_memory=chat_mem_history, return_messages=True)
q = ConversationalRetrievalChain.from_llm(
    llm=chat,
    memory=mem,
    retriever=neo4j_vector.as_retriever()
)

In [25]:
response = q({'question': 'Podcast'})
response.get('answer')

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 31462 tokens. Please reduce the length of the messages.